# Training data generation

This notebook takes the text database from past editions of the [Survey on Inflation and Growth Expectations](https://www.bancaditalia.it/pubblicazioni/indagine-inflazione/index.html) by Banca d'Italia in order to generate synthetic data for the fine tuning of a Large Language Model.

In order to do so, we leverage the free tier of a few cloud services which provide APIs for LLM use.

## Load libraries and data

In [1]:
import time
from datetime import datetime

import pandas as pd
import dashscope
from cerebras.cloud.sdk import Cerebras
from mistralai import Mistral


In [2]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [3]:
text_df = pd.read_excel("data/testi_IAI.xlsx")

In [4]:
text_dicts = text_df.to_dict(orient="records")

In [5]:
system_prompt = """Sei un economista che sta scrivendo una prima bozza di un documento. Ti verrà fornito un paragrafo di un testo economico nella sua versione finale e pubblicata dalla Banca d'Italia.

Il tuo compito è generare una possibile versione precedente di questo testo - una bozza grezza che avrebbe potuto essere scritta prima delle revisioni e correzioni finali.

La bozza che genererai dovrà contenere:
- Ripetizioni di concetti o parole
- Abbreviazioni tipiche di un documento di lavoro
- Semplificazioni eccessive di concetti economici
- Frasi più lunghe e meno strutturate
- Qualche imprecisione terminologica
- Un linguaggio meno formale e più colloquiale
- Possibili ridondanze

Questa bozza dovrà comunque mantenere i concetti fondamentali e i dati esatti del testo originale, ma presentandoli in forma meno raffinata e professionale.
Non aggiungere informazioni non presenti nel testo.
La tua risposta deve contenere solamente la possibile bozza precedente, senza alcun altro testo o informazione.
"""

prompt_template = """Paragrafo nella versione finale:
{testo}

Genera una possibile bozza precedente di questo paragrafo:
"""

## Alibaba Cloud

In [6]:
dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

In [7]:
qwen_results = []

for item in text_dicts:
    try:
        time.sleep(3) # Respect API limits
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt_template.format(testo=item.get("testo"))}
        ]
        response = dashscope.Generation.call(
            api_key=config.get('ALIBABA_API_KEY'),
            model="qwen2.5-72b-instruct", 
            messages=messages,
            result_format='message'
            )

        qwen_results.append({**item, "input": response.output.choices[0].message.content})
    except:
        continue

In [8]:
qwen_df = pd.DataFrame(qwen_results)
qwen_df.to_excel("data/testi_qwen_{}.xlsx".format(datetime.now().strftime("%Y%m%d")), index=False)

## Mistral

In [9]:
mistral_model = "mistral-large-latest"

mistral_client = Mistral(api_key=config.get('MISTRAL_API_KEY'))

In [10]:
mistral_results = []

for item in text_dicts:
    try:
        time.sleep(3) # Respect API limits
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt_template.format(testo=item.get("testo"))}
        ]
        response = mistral_client.chat.complete(
            model= mistral_model,
            messages = messages
        )

        mistral_results.append({**item, "input": response.choices[0].message.content})
    except:
        continue

In [11]:
mistral_df = pd.DataFrame(mistral_results)
mistral_df.to_excel("data/testi_mistral_{}.xlsx".format(datetime.now().strftime("%Y%m%d")), index=False)

## Cerebras

In [12]:
cerebras_client = Cerebras(
  api_key=config.get('CEREBRAS_API_KEY'),
)

In [13]:
cerebras_results = []

for item in text_dicts:
    try:
        time.sleep(3) # Respect API limits
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': prompt_template.format(testo=item.get("testo"))}
        ]
        response = cerebras_client.chat.completions.create(
            messages=messages,
            model="llama-4-scout-17b-16e-instruct",
        )

        cerebras_results.append({**item, "input": response.choices[0].message.content})
    except:
        continue

In [14]:
cerebas_df = pd.DataFrame(cerebras_results)
cerebas_df.to_excel("data/testi_cerebras_{}.xlsx".format(datetime.now().strftime("%Y%m%d")), index=False)